In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

import pickle


In [126]:
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [3]:
# Part-Of-Speech Tagging
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [4]:
df = pd.read_csv("wines_cleaned_150k.csv")
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [5]:
variety_unique = list(df["variety"].unique())
variety_unique


['Cabernet Sauvignon',
 'Tinta de Toro',
 'Sauvignon Blanc',
 'Pinot Noir',
 'Provence red blend',
 'Friulano',
 'Tannat',
 'Chardonnay',
 'Tempranillo',
 'Malbec',
 'Rosé',
 'Tempranillo Blend',
 'Syrah',
 'Mavrud',
 'Sangiovese',
 'Sparkling Blend',
 'Rhône-style White Blend',
 'Red Blend',
 'Mencía',
 'Palomino',
 'Petite Sirah',
 'Riesling',
 'Cabernet Sauvignon-Syrah',
 'Portuguese Red',
 'Nebbiolo',
 'Pinot Gris',
 'Meritage',
 'Baga',
 'Glera',
 'Malbec-Merlot',
 'Merlot-Malbec',
 'Ugni Blanc-Colombard',
 'Viognier',
 'Cabernet Sauvignon-Cabernet Franc',
 'Moscato',
 'Pinot Grigio',
 'Cabernet Franc',
 'White Blend',
 'Monastrell',
 'Gamay',
 'Zinfandel',
 'Greco',
 'Barbera',
 'Grenache',
 'Rhône-style Red Blend',
 'Albariño',
 'Malvasia Bianca',
 'Assyrtiko',
 'Malagouzia',
 'Carmenère',
 'Bordeaux-style Red Blend',
 'Touriga Nacional',
 'Agiorgitiko',
 'Picpoul',
 'Godello',
 'Gewürztraminer',
 'Merlot',
 'Syrah-Grenache',
 'G-S-M',
 'Mourvèdre',
 'Bordeaux-style White Blend'

In [6]:
variety_df = pd.DataFrame(variety_unique)
variety_df.to_csv("wine_variety.csv")

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150930 entries, 0 to 150929
Data columns (total 11 columns):
Unnamed: 0     150930 non-null int64
country        150930 non-null object
description    150930 non-null object
designation    105195 non-null object
points         150930 non-null int64
price          137235 non-null float64
province       150926 non-null object
region_1       125870 non-null object
region_2       60953 non-null object
variety        150930 non-null object
winery         150930 non-null object
dtypes: float64(1), int64(2), object(8)
memory usage: 12.7+ MB


In [8]:
df.describe()

,Unnamed: 0,points,price
count,150930.000000,150930.000000,137235.000000
mean,75464.500000,87.888418,33.131482
std,43569.882402,3.222392,36.322536
min,0.000000,80.000000,4.000000
25%,37732.250000,86.000000,16.000000
50%,75464.500000,88.000000,24.000000
75%,113196.750000,90.000000,40.000000
max,150929.000000,100.000000,2300.000000


In [9]:
df1 = pd.read_csv("wine_variety_complete.csv")


In [10]:
df1

,Unnamed: 0,variety,type
0,0,Cabernet Sauvignon,Red
1,1,Tinta de Toro,Red
2,2,Sauvignon Blanc,White
3,3,Pinot Noir,Red
4,4,Provence red blend,Red
5,5,Friulano,White
6,6,Tannat,Red
7,7,Chardonnay,White
8,8,Tempranillo,Red
9,9,Malbec,Red


In [11]:
merged_df = pd.merge(df, df1, on = "variety", how = "outer")

In [12]:
merged_df

,Unnamed: 0_x,country,description,designation,points,price,province,region_1,region_2,variety,winery,Unnamed: 0_y,type
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,0,Red
1,16,US,"This blockbuster, powerhouse of a wine suggest...",Rainin Vineyard,95,325.0,California,Diamond Mountain District,Napa,Cabernet Sauvignon,Hall,0,Red
2,47,US,"Blended with 9% Malbec, 9% Cabernet Franc and ...",Estate Grown,90,60.0,California,Mount Veeder,Napa,Cabernet Sauvignon,Brandlin,0,Red
3,49,US,"The aromas of blue fruit, herbs and spice are ...",NaN,90,40.0,Washington,Red Mountain,Columbia Valley,Cabernet Sauvignon,Canvasback,0,Red
4,55,US,"A blend of Cabernet from Grand Ciel (31%), Cie...",Four Flags,90,69.0,Washington,Red Mountain,Columbia Valley,Cabernet Sauvignon,DeLille,0,Red
5,67,US,From the producer's monumental Atlas Peak vine...,Animo,91,85.0,California,Napa Valley,Napa,Cabernet Sauvignon,Michael Mondavi Family Estate,0,Red
6,69,US,A juiciness of cherry and vanilla spark the op...,Barrel Select,91,60.0,California,Rutherford,Napa,Cabernet Sauvignon,Provenance Vineyards,0,Red
7,70,US,Sweetened tannins highlight a depth of chocola...,District Collection,91,85.0,California,St. Helena,Napa,Cabernet Sauvignon,Raymond,0,Red
8,78,US,This wine over-delivers in quality for the pri...,NaN,91,28.0,California,Napa Valley,Napa,Cabernet Sauvignon,B Side,0,Red
9,81,Israel,"Dark red violet in color, this wine has aromas...",Reserve,91,25.0,Upper Galilee,NaN,NaN,Cabernet Sauvignon,Binyamina,0,Red


In [13]:
merged_df.head()

,Unnamed: 0_x,country,description,designation,points,price,province,region_1,region_2,variety,winery,Unnamed: 0_y,type
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,0,Red
1,16,US,"This blockbuster, powerhouse of a wine suggest...",Rainin Vineyard,95,325.0,California,Diamond Mountain District,Napa,Cabernet Sauvignon,Hall,0,Red
2,47,US,"Blended with 9% Malbec, 9% Cabernet Franc and ...",Estate Grown,90,60.0,California,Mount Veeder,Napa,Cabernet Sauvignon,Brandlin,0,Red
3,49,US,"The aromas of blue fruit, herbs and spice are ...",NaN,90,40.0,Washington,Red Mountain,Columbia Valley,Cabernet Sauvignon,Canvasback,0,Red
4,55,US,"A blend of Cabernet from Grand Ciel (31%), Cie...",Four Flags,90,69.0,Washington,Red Mountain,Columbia Valley,Cabernet Sauvignon,DeLille,0,Red


In [14]:
merged_df.drop("Unnamed: 0_x", axis=1, inplace=True)

In [15]:
merged_df.drop("Unnamed: 0_y", axis=1, inplace=True)

In [16]:
merged_df.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery,type
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,Red
1,US,"This blockbuster, powerhouse of a wine suggest...",Rainin Vineyard,95,325.0,California,Diamond Mountain District,Napa,Cabernet Sauvignon,Hall,Red
2,US,"Blended with 9% Malbec, 9% Cabernet Franc and ...",Estate Grown,90,60.0,California,Mount Veeder,Napa,Cabernet Sauvignon,Brandlin,Red
3,US,"The aromas of blue fruit, herbs and spice are ...",NaN,90,40.0,Washington,Red Mountain,Columbia Valley,Cabernet Sauvignon,Canvasback,Red
4,US,"A blend of Cabernet from Grand Ciel (31%), Cie...",Four Flags,90,69.0,Washington,Red Mountain,Columbia Valley,Cabernet Sauvignon,DeLille,Red


In [17]:
len(merged_df)

150930

In [18]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150930 entries, 0 to 150929
Data columns (total 11 columns):
country        150930 non-null object
description    150930 non-null object
designation    105195 non-null object
points         150930 non-null int64
price          137235 non-null float64
province       150926 non-null object
region_1       125870 non-null object
region_2       60953 non-null object
variety        150930 non-null object
winery         150930 non-null object
type           150930 non-null object
dtypes: float64(1), int64(1), object(9)
memory usage: 13.8+ MB


In [19]:
regression_df = merged_df[merged_df["price"] >0]

In [20]:
regression_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137235 entries, 0 to 150929
Data columns (total 11 columns):
country        137235 non-null object
description    137235 non-null object
designation    94924 non-null object
points         137235 non-null int64
price          137235 non-null float64
province       137231 non-null object
region_1       114393 non-null object
region_2       60704 non-null object
variety        137235 non-null object
winery         137235 non-null object
type           137235 non-null object
dtypes: float64(1), int64(1), object(9)
memory usage: 12.6+ MB


In [21]:
regression_df.describe()

,points,price
count,137235.000000,137235.000000
mean,87.787919,33.131482
std,3.221680,36.322536
min,80.000000,4.000000
25%,86.000000,16.000000
50%,88.000000,24.000000
75%,90.000000,40.000000
max,100.000000,2300.000000


In [22]:
regression_df.corr()

,points,price
points,1.000000,0.459863
price,0.459863,1.000000


In [23]:
regression_df.drop("designation", axis=1, inplace=True)

C:\Users\falcon\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [24]:
regression_df.drop("province", axis=1, inplace=True)

C:\Users\falcon\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [25]:
regression_df.head()

,country,description,points,price,region_1,region_2,variety,winery,type
0,US,This tremendous 100% varietal wine hails from ...,96,235.0,Napa Valley,Napa,Cabernet Sauvignon,Heitz,Red
1,US,"This blockbuster, powerhouse of a wine suggest...",95,325.0,Diamond Mountain District,Napa,Cabernet Sauvignon,Hall,Red
2,US,"Blended with 9% Malbec, 9% Cabernet Franc and ...",90,60.0,Mount Veeder,Napa,Cabernet Sauvignon,Brandlin,Red
3,US,"The aromas of blue fruit, herbs and spice are ...",90,40.0,Red Mountain,Columbia Valley,Cabernet Sauvignon,Canvasback,Red
4,US,"A blend of Cabernet from Grand Ciel (31%), Cie...",90,69.0,Red Mountain,Columbia Valley,Cabernet Sauvignon,DeLille,Red


In [26]:
regression_df.drop("region_1", axis=1, inplace=True)

C:\Users\falcon\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [27]:
regression_df.drop("region_2", axis = 1, inplace=True)

C:\Users\falcon\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [28]:
regression_df.drop("winery", axis = 1, inplace = True)

C:\Users\falcon\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [29]:
regression_df.head()

,country,description,points,price,variety,type
0,US,This tremendous 100% varietal wine hails from ...,96,235.0,Cabernet Sauvignon,Red
1,US,"This blockbuster, powerhouse of a wine suggest...",95,325.0,Cabernet Sauvignon,Red
2,US,"Blended with 9% Malbec, 9% Cabernet Franc and ...",90,60.0,Cabernet Sauvignon,Red
3,US,"The aromas of blue fruit, herbs and spice are ...",90,40.0,Cabernet Sauvignon,Red
4,US,"A blend of Cabernet from Grand Ciel (31%), Cie...",90,69.0,Cabernet Sauvignon,Red


In [30]:
len(regression_df)

137235

In [31]:
from textblob import TextBlob

In [32]:
reviews = regression_df["description"]

In [33]:
polarity = []
subjectivity = []
for review in reviews:
    testimonial = TextBlob(review)
    pol = testimonial.sentiment.polarity
    polarity.append(pol)
    sen = testimonial.sentiment.subjectivity
    subjectivity.append(sen)

In [34]:
my_dict = {"description": reviews, "polarity": polarity, "subjectivity": subjectivity}
my_dict

{'description': 0         This tremendous 100% varietal wine hails from ...
 1         This blockbuster, powerhouse of a wine suggest...
 2         Blended with 9% Malbec, 9% Cabernet Franc and ...
 3         The aromas of blue fruit, herbs and spice are ...
 4         A blend of Cabernet from Grand Ciel (31%), Cie...
 5         From the producer's monumental Atlas Peak vine...
 6         A juiciness of cherry and vanilla spark the op...
 7         Sweetened tannins highlight a depth of chocola...
 8         This wine over-delivers in quality for the pri...
 9         Dark red violet in color, this wine has aromas...
 10        From a multiplicity of vineyards, this wine wo...
 11        Bright, light oak shadings dress up this mediu...
 12        Blended with small amounts of Merlot, Cabernet...
 13        Brambly and herbal, this wine shows bright aci...
 14        This mountain-grown wine is hugely tannic and ...
 15        This is a thick, chocolatey wine with a side o...
 16      

In [35]:
textblob_df = pd.DataFrame(my_dict)
textblob_df.head()

,description,polarity,subjectivity
0,This tremendous 100% varietal wine hails from ...,2.240741e-01,0.666667
1,"This blockbuster, powerhouse of a wine suggest...",9.251859e-18,0.466667
2,"Blended with 9% Malbec, 9% Cabernet Franc and ...",2.222222e-01,0.255556
3,"The aromas of blue fruit, herbs and spice are ...",1.554422e-01,0.347959
4,"A blend of Cabernet from Grand Ciel (31%), Cie...",4.666667e-01,0.900000


In [36]:
len(textblob_df)

137235

In [37]:
textblob_df["polarity"]= textblob_df["polarity"].astype(float)

In [38]:
textblob_df["subjectivity"] = textblob_df["subjectivity"].astype(float)

In [39]:
textblob_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137235 entries, 0 to 150929
Data columns (total 3 columns):
description     137235 non-null object
polarity        137235 non-null float64
subjectivity    137235 non-null float64
dtypes: float64(2), object(1)
memory usage: 4.2+ MB


In [40]:
textblob_df.head()

,description,polarity,subjectivity
0,This tremendous 100% varietal wine hails from ...,2.240741e-01,0.666667
1,"This blockbuster, powerhouse of a wine suggest...",9.251859e-18,0.466667
2,"Blended with 9% Malbec, 9% Cabernet Franc and ...",2.222222e-01,0.255556
3,"The aromas of blue fruit, herbs and spice are ...",1.554422e-01,0.347959
4,"A blend of Cabernet from Grand Ciel (31%), Cie...",4.666667e-01,0.900000


In [41]:
textblob_df.round(3)

,description,polarity,subjectivity
0,This tremendous 100% varietal wine hails from ...,0.224,0.667
1,"This blockbuster, powerhouse of a wine suggest...",0.000,0.467
2,"Blended with 9% Malbec, 9% Cabernet Franc and ...",0.222,0.256
3,"The aromas of blue fruit, herbs and spice are ...",0.155,0.348
4,"A blend of Cabernet from Grand Ciel (31%), Cie...",0.467,0.900
5,From the producer's monumental Atlas Peak vine...,-0.036,0.305
6,A juiciness of cherry and vanilla spark the op...,0.400,0.500
7,Sweetened tannins highlight a depth of chocola...,0.117,0.583
8,This wine over-delivers in quality for the pri...,0.275,0.494
9,"Dark red violet in color, this wine has aromas...",0.164,0.431


In [42]:
textblob_df.drop("description", axis=1, inplace=True)

In [43]:
textblob_df.head()

,polarity,subjectivity
0,2.240741e-01,0.666667
1,9.251859e-18,0.466667
2,2.222222e-01,0.255556
3,1.554422e-01,0.347959
4,4.666667e-01,0.900000


In [44]:
# regression_df.append(textblob_df)

In [45]:
merged_df = pd.concat([regression_df, textblob_df], axis=1)

In [46]:
merged_df.round(3)

,country,description,points,price,variety,type,polarity,subjectivity
0,US,This tremendous 100% varietal wine hails from ...,96,235.0,Cabernet Sauvignon,Red,0.224,0.667
1,US,"This blockbuster, powerhouse of a wine suggest...",95,325.0,Cabernet Sauvignon,Red,0.000,0.467
2,US,"Blended with 9% Malbec, 9% Cabernet Franc and ...",90,60.0,Cabernet Sauvignon,Red,0.222,0.256
3,US,"The aromas of blue fruit, herbs and spice are ...",90,40.0,Cabernet Sauvignon,Red,0.155,0.348
4,US,"A blend of Cabernet from Grand Ciel (31%), Cie...",90,69.0,Cabernet Sauvignon,Red,0.467,0.900
5,US,From the producer's monumental Atlas Peak vine...,91,85.0,Cabernet Sauvignon,Red,-0.036,0.305
6,US,A juiciness of cherry and vanilla spark the op...,91,60.0,Cabernet Sauvignon,Red,0.400,0.500
7,US,Sweetened tannins highlight a depth of chocola...,91,85.0,Cabernet Sauvignon,Red,0.117,0.583
8,US,This wine over-delivers in quality for the pri...,91,28.0,Cabernet Sauvignon,Red,0.275,0.494
9,Israel,"Dark red violet in color, this wine has aromas...",91,25.0,Cabernet Sauvignon,Red,0.164,0.431


In [47]:
merged_df = merged_df.round(3)

In [48]:
merged_df.head()

,country,description,points,price,variety,type,polarity,subjectivity
0,US,This tremendous 100% varietal wine hails from ...,96,235.0,Cabernet Sauvignon,Red,0.224,0.667
1,US,"This blockbuster, powerhouse of a wine suggest...",95,325.0,Cabernet Sauvignon,Red,0.000,0.467
2,US,"Blended with 9% Malbec, 9% Cabernet Franc and ...",90,60.0,Cabernet Sauvignon,Red,0.222,0.256
3,US,"The aromas of blue fruit, herbs and spice are ...",90,40.0,Cabernet Sauvignon,Red,0.155,0.348
4,US,"A blend of Cabernet from Grand Ciel (31%), Cie...",90,69.0,Cabernet Sauvignon,Red,0.467,0.900


In [49]:
# creating a function to take in the description(review) and preprocess it.
def preprocess(sent):
    sent = word_tokenize(sent)
    sent = pos_tag(sent)
    return sent

In [50]:
des = []
views = merged_df["description"]

for view in views:
    prep_data = preprocess(view)
    des.append(prep_data)

In [67]:
review_words = des
des_df = pd.DataFrame(review_words)
des_df.head()


,0,1,2,3,4,5,6,7,8,9,...,146,147,148,149,150,151,152,153,154,155
0,"(This, DT)","(tremendous, JJ)","(100, CD)","(%, NN)","(varietal, JJ)","(wine, NN)","(hails, NNS)","(from, IN)","(Oakville, NNP)","(and, CC)",...,None,None,None,None,None,None,None,None,None,None
1,"(This, DT)","(blockbuster, NN)","(,, ,)","(powerhouse, NN)","(of, IN)","(a, DT)","(wine, JJ)","(suggests, VBZ)","(blueberry, VBP)","(pie, NN)",...,None,None,None,None,None,None,None,None,None,None
2,"(Blended, VBN)","(with, IN)","(9, CD)","(%, NN)","(Malbec, NNP)","(,, ,)","(9, CD)","(%, NN)","(Cabernet, NNP)","(Franc, NNP)",...,None,None,None,None,None,None,None,None,None,None
3,"(The, DT)","(aromas, NN)","(of, IN)","(blue, JJ)","(fruit, NN)","(,, ,)","(herbs, NN)","(and, CC)","(spice, NN)","(are, VBP)",...,None,None,None,None,None,None,None,None,None,None
4,"(A, DT)","(blend, NN)","(of, IN)","(Cabernet, NNP)","(from, IN)","(Grand, NNP)","(Ciel, NNP)","((, ()","(31, CD)","(%, NN)",...,None,None,None,None,None,None,None,None,None,None


In [68]:
len(des_df)

137235

In [84]:
count_words = des_df.groupby([0]).count()
count_words

,1,2,3,4,5,6,7,8,9,10,...,146,147,148,149,150,151,152,153,154,155
0,,,,,,,,,,,,,,,,,,,,,
"($, $)",4,4,4,4,4,4,4,4,4,4,...,0,0,0,0,0,0,0,0,0,0
"(), ))",7,7,7,7,7,7,7,7,7,7,...,0,0,0,0,0,0,0,0,0,0
"(., .)",40,40,40,40,40,40,40,40,40,40,...,0,0,0,0,0,0,0,0,0,0
"(100, CD)",30,30,30,30,30,30,30,30,30,30,...,0,0,0,0,0,0,0,0,0,0
"(105, CD)",2,2,2,2,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
"(1997, CD)",1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
"(2008, CD)",1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
"(2010, CD)",1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
"(2011, CD)",1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [85]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [202]:
# extract length of review
merged_df["length"] = merged_df["description"].map(lambda x: len(x))

In [205]:
merged_df.head()

,country,description,points,price,variety,type,polarity,subjectivity,length
0,US,This tremendous 100% varietal wine hails from ...,96,235.0,Cabernet Sauvignon,Red,0.224,0.667,355
1,US,"This blockbuster, powerhouse of a wine suggest...",95,325.0,Cabernet Sauvignon,Red,0.000,0.467,276
2,US,"Blended with 9% Malbec, 9% Cabernet Franc and ...",90,60.0,Cabernet Sauvignon,Red,0.222,0.256,293
3,US,"The aromas of blue fruit, herbs and spice are ...",90,40.0,Cabernet Sauvignon,Red,0.155,0.348,177
4,US,"A blend of Cabernet from Grand Ciel (31%), Cie...",90,69.0,Cabernet Sauvignon,Red,0.467,0.900,225


In [219]:
# define x and y
x = merged_df["description"]
y = merged_df["price"]

# split the df into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state=42)

In [220]:
print(x)

0         This tremendous 100% varietal wine hails from ...
1         This blockbuster, powerhouse of a wine suggest...
2         Blended with 9% Malbec, 9% Cabernet Franc and ...
3         The aromas of blue fruit, herbs and spice are ...
4         A blend of Cabernet from Grand Ciel (31%), Cie...
5         From the producer's monumental Atlas Peak vine...
6         A juiciness of cherry and vanilla spark the op...
7         Sweetened tannins highlight a depth of chocola...
8         This wine over-delivers in quality for the pri...
9         Dark red violet in color, this wine has aromas...
10        From a multiplicity of vineyards, this wine wo...
11        Bright, light oak shadings dress up this mediu...
12        Blended with small amounts of Merlot, Cabernet...
13        Brambly and herbal, this wine shows bright aci...
14        This mountain-grown wine is hugely tannic and ...
15        This is a thick, chocolatey wine with a side o...
16        While Cabernet Sauvignon domin

In [169]:
len(x)

137235

In [170]:
len(x_test)/len(x_train)

0.25

In [171]:
# TF-IDF in our model and Stop Words
vect = TfidfVectorizer(stop_words='english')
x_train_dtm = vect.fit_transform(x_train)
x_test_dtm = vect.transform(x_test)
features = vect.get_feature_names()

# save the  vectorizer to disk
filename = 'vectorizer.sav'
pickle.dump(vect, open(filename, 'wb'))


# print(x_train_dtm.shape)
# print(x_test_dtm.shape)

In [172]:
# # Term Frequency

# tf = pd.DataFrame(dtm.toarray(), columns=vect.get_feature_names())

In [222]:
# # defining the features (x) and output (y) columns
col = [ "points", "type", "polarity", "subjectivity", "length"]
x = merged_df[col]
y = merged_df["price"]

In [223]:
# converting categories to dummies
x = pd.get_dummies(x, columns = [ "type"])

# save the  dummies to disk
filename = 'dummies.sav'
pickle.dump(x, open(filename, 'wb'))

In [175]:
x.head()

,points,polarity,subjectivity,length,type_Brown,type_Red,type_Red,type_Rose,type_White
0,96,0.224,0.667,355,0,1,0,0,0
1,95,0.000,0.467,276,0,1,0,0,0
2,90,0.222,0.256,293,0,1,0,0,0
3,90,0.155,0.348,177,0,1,0,0,0
4,90,0.467,0.900,225,0,1,0,0,0


In [176]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137235 entries, 0 to 150929
Data columns (total 9 columns):
points          137235 non-null int64
polarity        137235 non-null float64
subjectivity    137235 non-null float64
length          137235 non-null int64
type_Brown      137235 non-null uint8
type_Red        137235 non-null uint8
type_Red        137235 non-null uint8
type_Rose       137235 non-null uint8
type_White      137235 non-null uint8
dtypes: float64(2), int64(2), uint8(5)
memory usage: 5.9 MB


In [177]:
merged_df.head()

,country,description,points,price,variety,type,polarity,subjectivity,length
0,US,This tremendous 100% varietal wine hails from ...,96,235.0,Cabernet Sauvignon,Red,0.224,0.667,355
1,US,"This blockbuster, powerhouse of a wine suggest...",95,325.0,Cabernet Sauvignon,Red,0.000,0.467,276
2,US,"Blended with 9% Malbec, 9% Cabernet Franc and ...",90,60.0,Cabernet Sauvignon,Red,0.222,0.256,293
3,US,"The aromas of blue fruit, herbs and spice are ...",90,40.0,Cabernet Sauvignon,Red,0.155,0.348,177
4,US,"A blend of Cabernet from Grand Ciel (31%), Cie...",90,69.0,Cabernet Sauvignon,Red,0.467,0.900,225


In [178]:
x.head()

,points,polarity,subjectivity,length,type_Brown,type_Red,type_Red,type_Rose,type_White
0,96,0.224,0.667,355,0,1,0,0,0
1,95,0.000,0.467,276,0,1,0,0,0
2,90,0.222,0.256,293,0,1,0,0,0
3,90,0.155,0.348,177,0,1,0,0,0
4,90,0.467,0.900,225,0,1,0,0,0


In [179]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [180]:
 x_train.head()
len(x_train)
len(x_test)


27447

In [181]:
# ### label encode the categorical values and convert them to numbers 
# le = LabelEncoder()
# le.fit(x_train['country'].astype(str))
# x_train['country'] = le.transform(x_train['country'].astype(str))
# x_test['country'] = le.transform(x_test['country'].astype(str))

# le.fit(x_train['type'].astype(str))
# x_train['type'] = le.transform(x_train['type'].astype(str))
# x_test['type'] = le.transform(x_test['type'].astype(str))

In [182]:
len(x)

137235

In [183]:
x_train

,points,polarity,subjectivity,length,type_Brown,type_Red,type_Red,type_Rose,type_White
23406,94,0.090,0.600,350,0,1,0,0,0
78297,87,0.169,0.458,333,0,1,0,0,0
144872,89,0.200,0.167,245,0,0,0,0,1
129675,80,0.127,0.458,196,0,0,0,0,1
123644,81,-0.211,0.597,122,0,1,0,0,0
73404,84,-0.139,0.369,282,0,1,0,0,0
49806,92,0.292,0.567,270,0,1,0,0,0
60796,93,0.280,0.545,311,0,1,0,0,0
31385,92,0.260,0.360,324,0,1,0,0,0
56309,87,0.383,0.693,216,0,0,0,1,0


In [184]:
y_train

23406      44.0
78297      30.0
144872     24.0
129675     12.0
123644     30.0
73404      18.0
49806      58.0
60796      75.0
31385      45.0
56309      15.0
8482       50.0
13966      13.0
97792      14.0
52049      50.0
19586      36.0
51620      30.0
146387     60.0
98396      47.0
52527      13.0
123897     50.0
82452      16.0
130732     25.0
77095      35.0
15373      17.0
31364      45.0
143529     10.0
79208     100.0
20203      55.0
111612     18.0
100564     10.0
          ...  
70600      27.0
130491     17.0
89558      35.0
142374     15.0
68840      30.0
135497     23.0
2775       19.0
142968     17.0
133932     25.0
89257      60.0
143078     15.0
90540      32.0
110070     19.0
5364       65.0
67968      30.0
62159      33.0
778        50.0
67838      18.0
70380      28.0
42667      26.0
16307      19.0
138426     28.0
122371     13.0
92988      12.0
57177      37.0
119866     12.0
131101     21.0
110491    120.0
144478     25.0
133348     19.0
Name: price, Length: 109

In [185]:
len(x_train)/len(x)

0.8

In [186]:
# cast other feature columns to float and convert to a sparse matrix
extra = sp.sparse.csr_matrix(x_train.astype(float))
extra.shape

(109788, 9)

In [187]:
# combine sparse matrices
x_train_dtm_extra = sp.sparse.hstack((x_train_dtm, extra))
x_train_dtm_extra.shape

(109788, 27663)

In [188]:
# repeat for testing set
# cast other feature columns to float and convert to a sparse matrix
extra_test = sp.sparse.csr_matrix(x_test.astype(float))
extra_test.shape

(27447, 9)

In [189]:
# combine sparse matrices

x_test_dtm_extra = sp.sparse.hstack((x_test_dtm, extra_test))
x_test_dtm_extra.shape


(27447, 27663)

In [190]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score

In [191]:
# define the model
model = LinearRegression()

# fit the model to training data
model.fit(x_train_dtm_extra, y_train)

# predict
y_train_pred = model.predict(x_train_dtm_extra)
y_test_pred = model.predict(x_test_dtm_extra)

# save the  model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [192]:
# r2 score tells how close you are to the correct 
print("The R^2 score for training data is", r2_score(y_train, y_train_pred))
print("The R^2 score for testing data is", r2_score(y_test, y_test_pred))

The R^2 score for training data is 0.639161204267831
The R^2 score for testing data is 0.21809686139132556


In [193]:
print("The train RMSE is", mean_squared_error(y_train, y_train_pred)**0.5)
print("The test RMSE is", mean_squared_error(y_test,y_test_pred)**0.5)

The train RMSE is 22.204264139221998
The test RMSE is 29.73990180451923


In [194]:
pd.DataFrame({"true_values": y_train, "predicted": y_train_pred,  "price_diff": y_train - y_train_pred})

,true_values,predicted,price_diff
23406,44.0,66.548776,-22.548776
78297,30.0,30.488778,-0.488778
144872,24.0,43.371840,-19.371840
129675,12.0,-1.685342,13.685342
123644,30.0,35.157912,-5.157912
73404,18.0,17.958161,0.041839
49806,58.0,45.761175,12.238825
60796,75.0,59.963054,15.036946
31385,45.0,54.101135,-9.101135
56309,15.0,9.998805,5.001195


In [114]:
# importing 3 different model
from sklearn.linear_model import Lasso, Ridge, ElasticNet

In [115]:
# define the model
lasso = Lasso(random_state = 42)

# fit the model to the data
lasso.fit(x_train_dtm_extra, y_train)

# predictions
y_pred_lasso = lasso.predict(x_test_dtm_extra)

RMSE_lasso = mean_squared_error(y_test, y_pred_lasso)**0.5
r2_lasso = r2_score(y_test, y_pred_lasso)

print(RMSE_lasso)
print(r2_lasso)

29.232611980663062
0.2445440606358572


In [116]:
# define the model
ridge = Ridge(random_state = 42)

# fit the model to the data
ridge.fit(x_train_dtm_extra, y_train)

# predictions
y_pred_ridge = ridge.predict(x_test_dtm_extra)

RMSE_ridge = mean_squared_error(y_test, y_pred_ridge)**0.5
r2_ridge = r2_score(y_test, y_pred_ridge)

print(RMSE_ridge)
print(r2_ridge)

30.575083469087936
0.1735640540944976


In [117]:
# define the model
en = ElasticNet(l1_ratio=0.20, random_state = 42)

# fit the model to the data
en.fit(x_train_dtm_extra, y_train)

# predictions
y_pred_en = en.predict(x_test_dtm_extra)

RMSE_en = mean_squared_error(y_test, y_pred_en)**0.5
r2_en = r2_score(y_test, y_pred_en)

print(RMSE_en)


29.2690965252503


In [118]:
# #KNeighborsRegressor model looks out for the values of the the nearest neighbors and gives out a average of the same

# from sklearn.neighbors import KNeighborsRegressor

# knn = KNeighborsRegressor(n_neighbors=5)

# knn.fit(x_train_dtm_extra, y_train)
# y_pred_knn = knn.predict(x_test_dtm_extra)

# RMSE_knn = mean_squared_error(y_test, y_pred_knn)**0.5
# r2_knn = r2_score(y_test, y_pred_knn)

# print(RMSE_knn)
# print(r2_knn)

In [119]:
# from sklearn.ensemble import RandomForestRegressor


In [120]:
# rf = RandomForestRegressor(n_estimators= 25, max_depth= None, max_features = 0.4, random_state = 42)
# rfmodel = rf.fit(X_train_dtm_extra, y_train)

# y_predrf = rfmodel.predict(X_test_dtm_extra)

# y_predrf

In [121]:
# rf = RandomForestRegressor(random_state = 42)
# rf.fit(x_train_dtm_extra[:100], y_train[:100])

# y_predrf = rf.predict(x_test_dtm_extra[:10])

# y_predrf